<a href="https://colab.research.google.com/github/forestsheart/Irradiancia-en-el-territorio-colombiano-una-perspectiva-espacial/blob/main/whisper-diarization-with-share-point-files.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
 upload audio file
from google.colab import files
uploaded = files.upload()
path = next(iter(uploaded))

Saving 1685637566.58332.mp3 to 1685637566.58332 (2).mp3


In [1]:
!pip install -q git+https://github.com/openai/whisper.git > /dev/null #libreria audio-to-text
!pip install -q git+https://github.com/pyannote/pyannote-audio > /dev/null #libreria, audio por segmento
!sudo apt update && sudo apt install ffmpeg #librerioa de manejo general de audio
!pip install pydub

Get:1 https://cloud.r-project.org/bin/linux/ubuntu focal-cran40/ InRelease [3,622 B]
Hit:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2004/x86_64  InRelease
Hit:3 http://archive.ubuntu.com/ubuntu focal InRelease
Get:4 http://archive.ubuntu.com/ubuntu focal-updates InRelease [114 kB]
Hit:5 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu focal InRelease
Get:6 http://security.ubuntu.com/ubuntu focal-security InRelease [114 kB]
Get:7 http://archive.ubuntu.com/ubuntu focal-backports InRelease [108 kB]
Get:8 https://cloud.r-project.org/bin/linux/ubuntu focal-cran40/ Packages [82.8 kB]
Hit:9 http://ppa.launchpad.net/cran/libgit2/ubuntu focal InRelease
Hit:10 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu focal InRelease
Get:11 http://archive.ubuntu.com/ubuntu focal-updates/main amd64 Packages [3,346 kB]
Hit:12 http://ppa.launchpad.net/graphics-drivers/ppa/ubuntu focal InRelease
Get:13 http://archive.ubuntu.com/ubuntu focal-updates/multiverse amd64 Packages [40.3 kB]
G

In [2]:
!pip install Office365-REST-Python-Client

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 909.2/909.2 kB 15.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 90.0/90.0 kB 13.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.3/4.3 MB 82.2 MB/s eta 0:00:00


In [3]:
#importamos libreria

import whisper #modelo audio to text
import datetime #trabajo formatos de fecha y tiempo

import subprocess #modificar archivos junto a ff

import torch #acelerador gpu
import pyannote.audio
from pyannote.audio.pipelines.speaker_verification import PretrainedSpeakerEmbedding #modelo pre-enmtrenado
embedding_model = PretrainedSpeakerEmbedding(
    "speechbrain/spkrec-ecapa-voxceleb",
    device=torch.device("cuda")) #modelo pre entrenado
#manejo audio
from pyannote.audio import Audio
from pyannote.core import Segment

import wave
import contextlib
#cluster to diarization
from sklearn.cluster import AgglomerativeClustering
import numpy as np

#conección share-point
from office365.runtime.auth.authentication_context import AuthenticationContext
from office365.sharepoint.client_context import ClientContext
import tempfile
import os

In [13]:
#definimos parametros del modelo
num_speakers = 2 #@param {type:"integer"}

language = 'Spanish' #@param ['any', 'English','Spanish']

model_size = 'medium' #@param ['tiny', 'base', 'small', 'medium', 'large']


model_name = model_size
if language == 'Spanish' and model_size != 'medium':
  model_name += '.es'

In [14]:
#definimos utf por defecto para trabajar en archivos .wav
import locale
def getpreferredencoding(do_setlocale = True):
    return "UTF-8"
locale.getpreferredencoding = getpreferredencoding

In [6]:
site_url = "https://refinancia-my.sharepoint.com/personal/bguerrerob_refinancia_co/"
username = "bguerrerob@refinancia.co"
password = "Rug28299"

ctx_auth = AuthenticationContext(site_url)
if ctx_auth.acquire_token_for_user(username, password):
    ctx = ClientContext(site_url, ctx_auth)
else:
    print("Error de autenticación")


In [15]:
model = whisper.load_model(model_size)

100%|█████████████████████████████████████| 1.42G/1.42G [00:23<00:00, 63.9MiB/s]


In [ ]:
file_url = "/personal/bguerrerob_refinancia_co/Documents/Documentos/Audios_junio/agent.12013.1014281519.date.2023-06-20-17-49-22.cod.001.tel.93117663535.id.12013.6211.1687301221.211264.hung_up.customer.mp3"
source_file = ctx.web.get_file_by_server_relative_url(file_url)


#source_file = ctx.web.get_file_by_server_relative_url(file_url)
local_file_name = os.path.join(tempfile.mkdtemp(), os.path.basename(file_url))
with open(local_file_name, "wb") as local_file:
    source_file.download_session(local_file).execute_query()
print("[Ok] file has been downloaded: {0}".format(local_file_name))

In [ ]:
path=local_file_name

In [ ]:
#si el archivo no es wav lo convertimos a
if path[-3:] != 'wav':
  subprocess.call(['ffmpeg', '-i', path, 'audio.wav', '-y'])
  path = 'audio.wav'

In [16]:
result = model.transcribe(path)
segments = result["segments"]

In [30]:
subprocess.call(['ffmpeg', '-i',path,  'output.wav'])
path="output.wav"

In [31]:
with contextlib.closing(wave.open(path,'r')) as f:
  frames = f.getnframes()
  rate = f.getframerate()
  duration = frames / float(rate)

In [32]:
audio = Audio()

def segment_embedding(segment):
  start = segment["start"]
  # Whisper overshoots the end timestamp in the last segment
  end = min(duration, segment["end"])
  clip = Segment(start, end)
  waveform, sample_rate = audio.crop(path, clip)
  return embedding_model(waveform[None])

In [33]:
embeddings = np.zeros(shape=(len(segments), 192))
for i, segment in enumerate(segments):
   embeddings[i] = segment_embedding(segment)

embeddings = np.nan_to_num(embeddings)

In [34]:
clustering = AgglomerativeClustering(num_speakers).fit(embeddings)
labels = clustering.labels_
for i in range(len(segments)):
  segments[i]["speaker"] = 'SPEAKER ' + str(labels[i] + 1)

In [35]:
def time(secs):
  return datetime.timedelta(seconds=round(secs))
f = open("transcript.txt", "w", encoding="utf-8")

for (i, segment) in enumerate(segments):
    if i == 0 or segments[i - 1]["speaker"] != segment["speaker"]:
        f.write("\n" + segment["speaker"] + ' ' + str(time(segment["start"])) + '\n')
    f.write(segment["text"][1:] + ' ')
f.close()


In [ ]:
###si el audio fuera 2 stereo seria mas facil y estable

In [5]:
output_cliente = "output_cliente.mp3"

ffmpeg_cmd_cliente = [
    "ffmpeg",
    "-i",
    local_file_name,
    "-ac",
    "1",
    "-map_channel",
    "0.0.0",
    output_cliente
]

subprocess.run(ffmpeg_cmd_cliente)


In [29]:
!whisper "/content/output_cliente.mp3"   --language es --model medium


In [ ]:
output_agente = "output_agente.wav"

ffmpeg_cmd_agente = [
    "ffmpeg",
    "-i",
    local_file_name,
    "-ac",
    "1",
    "-map_channel",
    "0.0.1",
    output_agente
]

subprocess.run(ffmpeg_cmd_agente)

In [ ]:
!whisper "/content/output_agente.wav"   --language es --model medium